In [1]:
import pickle 

with open("final_list_yerke_vector_db_final.pkl", "rb") as f:
    mylist = pickle.load(f)

In [19]:
from fastembed import TextEmbedding
from qdrant_client import QdrantClient

Qclient = QdrantClient(url="http://localhost:6333")

In [20]:
from qdrant_client.models import Distance, VectorParams


collection_name = "YerkeChatBot"



if Qclient.collection_exists(collection_name=f"{collection_name}"):
    Qclient.delete_collection(collection_name=f"{collection_name}")
Qclient.create_collection(
    collection_name=f"{collection_name}",
    vectors_config=VectorParams(size=768, distance=Distance.COSINE),
)

True

In [25]:
Qclient.collection_exists(collection_name=f"{collection_name}")

True

In [22]:
model = TextEmbedding("nomic-ai/nomic-embed-text-v1")

Fetching 5 files: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 28416.69it/s]


In [26]:
from qdrant_client.models import PointStruct
import numpy as np 
from tqdm import tqdm 
mypoints = [] 
mypoints_save = []
for idx, eachList in enumerate(tqdm(mylist)):
    title = eachList['title']
    summary = eachList['summary']
    context = eachList["propositions"]
    embedding1 = next(model.embed([title]))
    embedding2 = next(model.embed([summary]))
    myvector = np.add(embedding1, embedding2)
    item = PointStruct(id=idx, vector = myvector.tolist(), payload={
        "summary": summary,
        "context": context})
    item_save = {"id": idx, "vector":  myvector.tolist(), 
                 "payload": {"summary": summary, "context": context}
                }
    mypoints.append(item)
    mypoints_save.append(item_save)



100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 695/695 [00:35<00:00, 19.33it/s]


In [27]:
from qdrant_client.models import PointStruct

operation_info = Qclient.upsert(
    collection_name=f"{collection_name}",
    wait=True,
    points=mypoints
)

print(operation_info)

operation_id=0 status=<UpdateStatus.COMPLETED: 'completed'>


In [32]:
query = "When will the Zoom broadcast be?"
query_vector = next(model.embed(query))

search_result = Qclient.search(
    collection_name=f"{collection_name}",
    query_vector=query_vector.tolist(), limit=1
)

In [33]:
search_result

[ScoredPoint(id=685, version=0, score=0.7818057, payload={'context': ['Zoom will be broadcast on Mondays from 20:00 to 21:00.', 'A second Zoom broadcast takes place on Thursdays between 21:00 and 22:00.', 'The Thursday Zoom broadcast is different every week.', 'The Thursday Zoom broadcast accommodates people with different schedules.', 'Participants will receive additional information on the Thursday Zoom broadcast.', 'The additional information about the Thursday Zoom broadcast does not affect the main lessons or homework.'], 'summary': 'This chunk contains information about the recurring Zoom broadcasts, including their schedule, weekly variations, and additional information for participants.'}, vector=None, shard_key=None, order_value=None)]

In [34]:
context = " ".join(search_result[0].payload["context"])

In [35]:
score = search_result[0].score

In [36]:
score

0.7818057

In [37]:
context

'Zoom will be broadcast on Mondays from 20:00 to 21:00. A second Zoom broadcast takes place on Thursdays between 21:00 and 22:00. The Thursday Zoom broadcast is different every week. The Thursday Zoom broadcast accommodates people with different schedules. Participants will receive additional information on the Thursday Zoom broadcast. The additional information about the Thursday Zoom broadcast does not affect the main lessons or homework.'

In [38]:
from openai import OpenAI

client = OpenAI(
    base_url='http://localhost:11434/v1/',
    api_key='ollama',
)

ModuleNotFoundError: No module named 'openai'

In [39]:
prompt_template = f"""
    Answer the "QUESTION" in detail, based on the "CONTEXT" provided. If context is empty, give answer as a financial advisor.
    QUESTION: 
    {query}
    CONTEXT: 
    {context}
    """.strip()
context = "\n".join(search_result[0].payload["context"]) if search_result[0].score >= 0.5 else ""
prompt = prompt_template.format(query=query, context=context).strip()

def llm(prompt):
    prompt_template = f"""
    Answer the "QUESTION" in detail, based on the "CONTEXT" provided. If context is empty, give answer as a financial advisor.
    QUESTION: 
    {query}
    CONTEXT: 
    {context}
    """.strip()
    context = "\n".join(search_result[0].payload["context"])
    prompt = prompt_template.format(query=query, context=context).strip()
    response = client.chat.completions.create(
        model='gemma2',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [ ]:
llm(prompt)

In [ ]:
import time
alls = []
for i in range(10):
    start_time = time.perf_counter()
    llm(prompt)
    end_time = time.perf_counter()
    elapsed_time = end_time - start_time
    print(f"Elapsed time: {elapsed_time :.6f} seconds")
    alls.append(elapsed_time)
    
print( "Average: ", sum(alls) / 10) 

In [ ]:
!pip list | grep sentence